<a href="https://colab.research.google.com/github/mba-fabio/dsa_usp/blob/main/20250420_Resultados_Preliminares_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Otimização da Distribuição de Joias e Relógios de Luxo Usados: Modelos Preditivos no mercado de luxo “Second Hand”

##Instalações e importações de pacotes

In [6]:
!pip install pulp

import pandas as pd
import pulp


## Importação de dados do Github

###Estoque Atual de cada loja

In [7]:
# Carrega os dados de estoque para o Data Frame do Pandas
estoque_joias = pd.read_csv('https://github.com/mba-fabio/dsa_usp/raw/main/20250330_estoque_joias.csv')
estoque_joias.info()
print(estoque_joias.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Loja          1900 non-null   object 
 1   Tipo de Joia  1900 non-null   object 
 2   Marca         1900 non-null   object 
 3   Metal         1900 non-null   object 
 4   Pedra         1900 non-null   object 
 5   Preço         1900 non-null   float64
dtypes: float64(1), object(5)
memory usage: 89.2+ KB
         Loja Tipo de Joia                Marca         Metal          Pedra  \
0  Shopping A     Pingente  Marca Internacional   ouro branco      sem pedra   
1  Shopping A        Colar            Sem Marca  ouro amarelo      esmeralda   
2  Shopping A     Pulseira            Sem Marca   ouro branco           rubi   
3  Shopping A     Pingente       Marca Nacional  ouro amarelo  outras pedras   
4  Shopping A       Brinco  Marca Internacional     ouro rosé      sem pedra   

    Preço  
0  1

###Histórico de vendas dos últimos 24 meses

In [8]:
# Carrega os dados de venda para o Data Frame do Pandas
vendas24m = pd.read_csv('https://github.com/mba-fabio/dsa_usp/raw/main/20250331_joias_simuladas_historico_24meses_sazonal.csv')
vendas24m.info()
print(vendas24m.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9560 entries, 0 to 9559
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Data              9560 non-null   object 
 1   Tipo_Joia         9560 non-null   object 
 2   Tipo_Ouro         9560 non-null   object 
 3   Tipo_Pedra        9560 non-null   object 
 4   Marca             9560 non-null   object 
 5   Localizacao_Loja  9560 non-null   object 
 6   Perfil_Cliente    9560 non-null   object 
 7   Preco             9560 non-null   float64
dtypes: float64(1), object(7)
memory usage: 597.6+ KB
      Data Tipo_Joia     Tipo_Ouro Tipo_Pedra                Marca  \
0  2022-01     Colar  Ouro Amarelo  Sem Pedra  Marca Internacional   
1  2022-01      Anel     Ouro Rosé  Sem Pedra  Marca Internacional   
2  2022-01    Brinco     Ouro Rosé  Sem Pedra  Marca Internacional   
3  2022-01  Corrente   Ouro Branco  Sem Pedra       Marca Nacional   
4  2022-01    Brinco

###Lote para distribuição entre lojas

In [9]:
# Carrega os dados do lote atual para o Data Frame do Pandas
lote = pd.read_csv('https://github.com/mba-fabio/dsa_usp/raw/main/20250405_Lote_Distribuicao_v3_desbalanceado.csv')
lote.info()
print(lote.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Tipo_Joia;Tipo_Ouro;Tipo_Pedra;Marca;Preco  110 non-null    object
dtypes: object(1)
memory usage: 1012.0+ bytes
          Tipo_Joia;Tipo_Ouro;Tipo_Pedra;Marca;Preco
0  Brinco;Ouro Amarelo;Sem Pedra;Marca Nacional;2...
1  Brinco;Ouro Branco;Sem Pedra;Marca Internacion...
2     Brinco;Ouro Amarelo;Rubi;Marca Nacional;5200.0
3  Brinco;Ouro Branco;Outras Pedras;Marca Interna...
4  Brinco;Ouro Amarelo;Diamante;Marca Nacional;83...


###Capacidade de estoque máxima de cada loja

In [10]:
# Carrega os dados de capacidade máxima de estoque de cada loja para o Data Frame do Pandas
capacidade_lojas = pd.read_csv('https://github.com/mba-fabio/dsa_usp/raw/main/20250419_Capacidade_Estoque_Lojas.csv')
capacidade_lojas.info()
print(capacidade_lojas.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   LOJA;ESTOQUE MAXIMO  4 non-null      object
dtypes: object(1)
memory usage: 164.0+ bytes
  LOJA;ESTOQUE MAXIMO
0      SHOPPING A;740
1      SHOPPING B;570
2      SHOPPING C;490
3      SHOPPING D;410
